# What is about 

Study degree distribution of KNN graphs for model data clouds - Gauss, uniform, sphere.
For high dimensions it seems to be  power law like. We also see drastical difference between sphere-torus vs Gauss-Uniform.
See discussion in : 
https://cstheory.stackexchange.com/questions/47957/power-law-for-degree-distribution-of-random-knn-graphs
see also https://stats.stackexchange.com/questions/500250/recognize-distribution-on-integers-0-mean-5-median-2-percentiles-75-5

V20 - same as 18,19 - but simulate Gauss, Uniform, Sphere, Torus 

V19 - same as 18, but trials increase to 100, and dim 1000 repeated twice with n_sample 1e4,1e5,

V18 - high dims explore: dim 100,1000,10000, K =5 n_sample = 1e4, Gauss 

V17 uniform , dim = 50, K = 10 

V16 same with , dim = 10, K = 20  strange thing  - mean sometimes not K for big sample size - is not a bug in cuml ? 
for dim 10 we see mean equals to median , while for dim 50 we saw mean was = percentile 75 

V15 same with , dim = 10, K = 10

V14 same with , dim = 10, K = 5

V13: K = 20

V12: repeat V11 for K = 10

V11 - study distribution in details for particular case - K=5, dim = 50 , change n_samples, look at percentiles, power law etc..  Findings summarized here: https://stats.stackexchange.com/questions/500250/recognize-distribution-on-integers-0-mean-5-median-2-percentiles-75-5

V10 - same as V9 - increase samples to 10_000_000 - crashed by memory after processing first case 

V9 - same as V8 - increase samples to 1_000_000 - 150 seconds 

V8 - simulations for Sphere and Torus added. See drastical difference for sphere-torus VS Gauss-uniform. 100_000 samples - 7 seconds 


V7 - same as V6 , but n_sample = 1e6 - run out of time 9 hours


V6 - linear interplotation of loglog plot - estimate exponent of power law of the distibution 

V5 - same as V4 but compare GPU and sklearn implementations - should coincide - indeed coincide 


V4 - pdf of degrees K=5, various dimensions , n_sample = 1e4

V3: plot pdf for :

Gauss dim50 n_neighbors3 n_sample100000 2276.7 2276.7 seconds passed trial, total

Gauss dim50 n_neighbors10 n_sample100000 2294.3 4571.1 seconds passed trial, total

V2: plot pdf for :

Gauss dim10 n_neighbors5 n_sample100000 73.2 73.2 seconds passed trial, total

Gauss dim50 n_neighbors5 n_sample100000 2313.5 2386.7 seconds passed trial, total

V1 - fast calculation:

Gauss dim10 n_neighbors5 n_sample10000 1.5 1.5 seconds passed trial, total

Gauss dim50 n_neighbors5 n_sample10000 11.9 13.4 seconds passed trial, total


dim10 n_neighbors5 n_sample1000 0.1 0.1 seconds passed trial, total

dim10 n_neighbors5 n_sample10000 1.5 1.6 seconds passed trial, total

dim10 n_neighbors5 n_sample100000 66.1 67.7 seconds passed trial, total

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.neighbors import kneighbors_graph
import time
import matplotlib.pyplot as plt

_t00 = time.time()

In [ ]:
try:
    import cudf
    from cuml.neighbors import NearestNeighbors
    #from cuml.datasets import make_blobs    
except:
    print('GPU library cannot be imported. Turn ON GPU')

In [ ]:
import numpy as np
from scipy.stats import rv_continuous
import scipy.integrate as integrate
import scipy.special as special
import plotly.graph_objs as go
from collections import Counter

class CustomDistribution(rv_continuous):
    def init(self, momtype = 'pdf', a=0, b=1):
        super().init(a=a, b=b)
    
    def _get_support(*args, **kwargs):
        return 0, 1
    
    def _pdf(self, x):
        if x < 0 or x > 1:
             return 0
            
        integral = integrate.quad(lambda t: np.sinh(t), 0, 1)[0]
        res = (np.sinh(x)) / integral
        return res

def scalar(x, y):
    return min(-1, (-x[0]*y[0] + x[1]*y[1] + x[2]*y[2]))

def metric(x, y):
    return np.arccosh(-scalar(x, y))

def get_data(distribution_type, dim, n_sample, R):
    if distribution_type == 'Gauss':
        X = np.random.randn(n_sample,dim)
    elif distribution_type == 'Sphere':
        X = np.random.randn(n_sample,dim+1)
        s = np.sqrt( (X*X).sum(axis=1))
        X = X / s[:,np.newaxis]
    elif distribution_type == 'Torus':
        X = ( np.random.rand(n_sample,dim) )
        X1 = np.sin(2*np.pi*X)
        X2 = np.cos(2*np.pi*X)
        X = np.concatenate((X1,X2),axis=1)
    elif distribution_type == 'hyperbolic':
        t0 = CustomDistribution().rvs(size=(n_sample, 1)) * R
        t1 = (np.random.rand(n_sample) * 2 * np.pi)
        t1 = t1.reshape(n_sample, 1)
        X0 = np.cosh(t0)
        X1 = np.sinh(t0) * np.cos(t1)
        X2 = np.sinh(t0) * np.sin(t1)
        X = np.concatenate((X0,X1,X2), axis=1)
        print('hyperbolic data is ready')
    else:
        X = np.random.uniform(-1,1, size = (n_sample,dim) )
    return X

    
        

In [ ]:
import collections
from scipy.stats import poisson
from scipy.stats import geom

def repeat_exp(distances,indices,num_iter):
    all_res = []
    all_edg = []
    for i in range(num_iter):
        graph =  {x: set() for x in range(len(indices))}
        inp_edg = {x: set() for x in range(len(indices))}
        roots = []
        for a, b in indices:
            if a < b and indices[b][1] == a:
                roots.append(a)
        for a, b in indices:
            graph[a].add(b)
            graph[b].add(a)
            inp_edg[b].add(a)

        all_edg += list(map(len, inp_edg.values()))
        res = bfs(graph, roots)
        all_res += res

#     connect(X, indices)
    return all_res, all_edg

def draw_comp_length(distances, indices):
    res = repeat_exp(distances, indices, 1)
    result, edges = res
    plot_hist(result, True)
    
def bfs(graph, roots): 
    sizes = []
    visited = set()
    for root in roots:
        if root not in visited:
            queue = collections.deque([root])
            visited.add(root)
            num = 1 
            while queue: 
                vertex = queue.popleft()
                for neighbour in graph[vertex]: 
                    if neighbour not in visited:
                        num += 1 
                        visited.add(neighbour) 
                        queue.append(neighbour)
            sizes.append(num)
    return sizes

def plot_hist(res, d):
    res = np.array(res)

    theta_geom = 1 / np.mean(res - 1)
    theta_poiss = np.mean(res - 2)
    plt.figure(figsize= (10, 10))
    rv = poisson(theta_poiss)
    #plt.hist(res, density = True, alpha = 0.7, bins = [i for i in range(2, max(res) + 1)])
    plot_list(res, d)
    x = np.arange(2, np.max(res) + 1)
    plt.plot(x, rv.pmf(x - 2), label='poisson', color = "red")
    rv = geom(theta_geom)
    plt.plot(x, rv.pmf(x - 1), label='geom', color = "green")
    plt.legend(loc='best', frameon=False)
    plt.show()
    
def plot_list(ar, density=False):
    dc = {}
    if type(ar) is list:
        st = set(ar)
        dc = dict.fromkeys(st, 0)
        for el in st:
            dc[el] = ar.count(el)
            if density:
                dc[el] /= len(ar)
    else:
        unique, counts = np.unique(ar, return_counts=True)
        if density:
            dc = dict(zip(unique, counts / len(ar)))
        else:
            dc = dict(zip(unique, counts))
#     plt.plot(list(dc.keys()), list(dc.values()), 'ob-', mfc='r')
    plt.semilogy(list(dc.keys()), list(dc.values()), 'ob-', mfc='r')
    s = sorted(dc.keys())
    prop = []
    for i in range(len(s) - 1):
        prop.append(dc[s[i]] / dc[s[i+1]])
    #plt.plot(prop)

def get_results():
    result = dict()
    edges = dict()
    for dim in [2, 5,]: # {2 : {100: [], 1000: []}}
        result[dim] = {100: [], 1000: []}
        edges[dim] = {100: [], 1000: []}
        for num_samples in result[dim]:
            result[dim][num_samples], edges[dim][num_samples] = repeat_exp(dim, num_samples, 25)
            print(f"Done dim {dim}, num_samples {num_samples}")
    return result, edges

def draw_graph(X, distances, indices):
    z, x, y = X.transpose()
    trace = go.Scatter3d(
       x = x, y = y, z = z,mode = 'markers', marker = dict(
          size = 3,
          color = z, # set color to an array/list of desired values
          colorscale = 'Viridis'
          )
       )
    
    layout = go.Layout(title = '3D Scatter plot')
    fig = go.Figure(data = [trace], layout = layout)
    for i, j in indices:
        z0, x0, y0 = X[i]
        z1, x1, y1 = X[j]
        x=np.linspace(x0, x1, 15)
        y=np.linspace(y0, y1, 15)
        z=np.linspace(z0, z1, 15)
        edge_trace = go.Scatter3d(x=x, y=y, z=z, mode='lines')
        fig.add_trace(edge_trace)
    fig.show()

In [ ]:
from sklearn.neighbors import NearestNeighbors

plt.style.use('ggplot')

fig = plt.figure(figsize=(20,12))

t00=time.time()
df = pd.DataFrame({'mean' : [],'std': []}) # mean/std table for each parameters set
list_save_results = []

n_sample0 = 100

for distribution_type in ['hyperbolic']:
    fig = plt.figure(figsize=(20,8))

    for R in [1, 5]:
        for dim , n_neighbors, n_sample in [
             (3, 1, n_sample0),
#              (3, 3, n_sample0),
#              (3, 5, n_sample0),

        ]:

            df_stat_over_trials = pd.DataFrame()
            all_degs = []

            for trial in range(2):
                t0=time.time()
                X = get_data(distribution_type, dim, int(n_sample), R)

                model = NearestNeighbors(n_neighbors=n_neighbors+1, algorithm='ball_tree', metric=metric)
                model.fit(X)

                # get 3 nearest neighbors
                m = model.kneighbors_graph(X)

                vec_degs = np.sum(m,axis = 0 )
                vec_degs = np.squeeze(np.array(vec_degs.ravel()))
                vec_degs -= np.ones_like(vec_degs ) # no include_self - so that is a way round
                all_degs.append(vec_degs)

            # Build histogram
            bins = np.arange(200)

            degs = bins[:-1]
            h = np.histogram(np.concatenate(all_degs), bins = bins, density=True)
            probs = h[0]
            m = h[0] > 1e-12

            label = f"{distribution_type} dim-{dim} n_neighbors-{n_neighbors} n_sample-{n_sample}"
            plt.plot( h[0][m],'*-', label = label, linewidth=4)
            plt.title(f'distribution of degrees R-{R}')
            plt.xlabel('degree')
            plt.ylabel('probability')
            plt.legend(fontsize = 12)
            d = {'mean': [np.concatenate(all_degs).mean()], 'std': [np.concatenate(all_degs).std()]}
            df1 = pd.DataFrame(data=d)
            df1.index = [f'{distribution_type} dim-{dim} n_neighbors-{n_neighbors} n_sample-{n_sample}']
            df = df.append(df1)

            if n_neighbors == 1:
                distances, indices = model.kneighbors(X)
                draw_comp_length(distances, indices)
                draw_graph(X, distances, indices)

    plt.show()
#     df.style.set_caption(f'R-{R}')
    print(df)    
seconds_passed_total = time.time() - t00
print( np.round(seconds_passed_total,1),  np.round(seconds_passed_total/60,1), np.round(seconds_passed_total/3600,1), 'seconds, minutes, hours passed')

